In [1]:
import sys
sys.path.append('../')
import os
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from instances.instance import *

problems are defined by: 
c, A, B (first stage jazz)

q_s, W_s, h_s, T_s, and clustering variables

In [2]:
gen = InvestmentGenerator('../stock_data.csv')
problem = gen.create_instance(20,200)

In [10]:
MP = gp.Model("MP")
MP.Params.outputFlag = 0
x = MP.addMVar((problem.s1_n_var,), name = "x")
eta = MP.addMVar((problem.k,), name = "eta", ub = problem.eta_bound, lb = -1*problem.eta_bound)

if problem.direction == GRB.MAXIMIZE:
    MP.modelSense = GRB.MAXIMIZE
else:
    MP.modelSense = GRB.MINIMIZE
MP.setObjective(
    problem.c @ x + np.array([1/problem.k] * problem.k) @ eta 
)
c1 = MP.addConstr(
    problem.A @ x <= problem.b
)

cut_found = True
n_iters = 0
try: 
    while cut_found:
        cut_found = False
        if n_iters < 200:
            n_iters = n_iters + 1
        else:
            break
        MP.update()
        MP.optimize()
        LB = 0
        x_i = x.x
        eta_i = eta.x
        for s in range(problem.k):
            SP = gp.Model("SP")
            #SP.Params.outputFlag = 0  # turn off output
            SP.Params.method = 1      # dual simplex
            y = SP.addMVar((problem.s2_n_var,), name = "y", obj = problem.q_list[s])
            SP.modelSense = GRB.MINIMIZE
            res = SP.addConstr(
                problem.W_list[s] @ y <= problem.h_list[s] + (problem.T_list[s] @ x_i)
            )
            SP.optimize()
            Q = SP.ObjVal
            pi = res.Pi
            LB = LB + Q / problem.k
        if np.abs(Q - eta_i[s]) < 0.00001:
            if Q - eta_i[s] < 0:
                cut_found = True
                p1 = pi @ problem.h_list[s]
                p2 = pi @ problem.T_list[s]
                MP.addConstr(
                    eta[s] <= p1 + gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                )
        print(LB)
except:
    print(s)


Set parameter Method to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 22 rows, 42 columns and 82 nonzeros
Model fingerprint: 0x8f3a8aaf
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 8e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
0


In [8]:
problem.s1_n_var

20

In [11]:
problem.q_list[0]

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1])